In [21]:
import os
_DF_DIR = os.path.join('..', 'analysis', 'df_files')
_CSV_DIR = os.path.join('..', 'analysis', 'csv_files')

import sys
sys.path.append(os.path.join('..', 'analysis', 'plots'))
sys.path.append(os.path.join('..', 'tools'))
import plotconfig

In [22]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.ticker import FuncFormatter
from scipy import stats
import pandas as pd
import numpy as np

In [23]:
dbf1 = os.path.join(_DF_DIR, 'results.parquet')
df1 = pd.read_parquet(dbf1, engine='pyarrow')

mask1 = ((df1['method_name'] == "LinearRegression") &
        (df1['eeg_name'] == "EEG_Raw") & 
        (df1['training_size'] == plotconfig.N_FOR_PERF_SCORE_COMPARISON) & 
        (df1['test_name'] == "diagonal"))

df_algorithm = df1[mask1]

In [24]:
humanfile = os.path.join(_CSV_DIR, 'image-distance-user-evaluations.csv')
df_human = pd.read_csv(humanfile)
df_human['round_d'] = round(df_human['distance'] * df_algorithm['true_distances'].explode().max(), 1)
human_unique_round_d = df_human['round_d'].unique()
human_unique_round_d

array([ 0. ,  1.6,  3.6,  6.8, 10.9, 16.3, 21.7, 29.4, 36.7, 46.2])

In [76]:
import dataset_tools
import file_tools
import face_tools
import saving_tools

_EXP_DIR = os.path.join('..', 'experiments')
_DATASET_DIR = os.path.join(_EXP_DIR, 'datasets')
_FACE_DIR = os.path.join('.', 'static')


In [86]:
from scipy.spatial import distance

faces_to_test = []
dataset_paths = file_tools.sort_filepaths([f for f in file_tools.list_folders(_DATASET_DIR)])
for dataset_path in dataset_paths:
        
    target_folder = os.path.join(dataset_path, 'target')
    target_filename = os.path.join(target_folder, 'target.npz')

    test_folder = os.path.join(dataset_path, 'test')
    test_filepaths = file_tools.sort_filepaths(file_tools.list_files(test_folder, 'diagonal.npz'))

    target_data = np.load(target_filename)
    target = target_data['target_face']

    for test_filename in test_filepaths:
        test_data = np.load(test_filename)
        test_faces = test_data['test_faces']

        for i, test_face in enumerate(test_faces):
            rounded_d = np.round(distance.euclidean(target, test_face), 1)
            if (rounded_d in human_unique_round_d):

                test_info = {}
                test_info["target_filename"] = target_filename
                test_info["test_filename"] = test_filename
                test_info["test_face_index"] = i
                test_info["rounded_d"] = rounded_d

                faces_to_test.append((target, test_face, test_info))
    

In [87]:
for i, (target, test_face, test_info) in enumerate(faces_to_test):

    save_foldername = os.path.join(_FACE_DIR, file_tools.generate_n_digit_name(i, n_digit=4))
    file_tools.ensure_dir(save_foldername)

    target_face_filename = os.path.join(save_foldername, 'target.jpg')
    target_face_img = face_tools.img_from_latent(target, size=1024)
    target_face_img.save(target_face_filename, 'JPEG')

    test_face_filename = os.path.join(save_foldername, 'test.jpg')
    test_face_img = face_tools.img_from_latent(test_face, size=1024)
    test_face_img.save(test_face_filename, 'JPEG')

    test_info_filename = os.path.join(save_foldername, 'info.json')
    saving_tools.save_dict_to_json(test_info, test_info_filename)